In [27]:
import time
import numpy as np
import py_at_broker as pab
from IPython.display import clear_output
import math as math
from toolbox import kinematic_mapping as km
import pandas as pd
import pylab as pl
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline
b = pab.broker()

In [28]:
b.register_signal('franka_target_pos', pab.MsgType.target_pos)

ZMQError: Address already in use

b.request_signal("franka_lidar", pab.MsgType.franka_lidar)

b.request_signal("franka_lidar", pab.MsgType.franka_lidar)
N = 100
lidar_data = np.zeros((N, 9))
for i in range(N): 
    lidar_msg = b.recv_msg("franka_lidar", 0)
    lid = lidar_msg.get_data()

idx = 2
lidar_data=[]

while True:
    lidar_msg = b.recv_msg("franka_lidar", -1)
    lid = lidar_msg.get_data()
    if (lid[idx]<3000): 
        lidar_data.append(lid[idx])
    plt.plot(lidar_data)
    plt.ylim(0, 1000)
    display.clear_output(wait=True)
    display.display(pl.gcf())
    time.sleep(0.1)

In [22]:
b.request_signal("franka_state", pab.MsgType.franka_state, True)

True

In [23]:
def create_spiral(t, a): 
    x = a*(np.cos(t) + t*np.sin(t))
    y = a*(np.sin(t) - t*np.cos(t))
    return x, y

In [24]:
def create_message(counter=1000, timestamp=time.CLOCK_MONOTONIC, \
                   ctrl_t=0, pos = np.array([0.5, 0.7, 0.7]), go_time=0.15):
    msg=pab.target_pos_msg()
    msg.set_fnumber(counter)
    msg.set_timestamp(timestamp)
    msg.set_ctrl_t(ctrl_t)
    msg.set_pos(pos)
    msg.set_time_to_go(go_time)
    current_counter=counter
    return msg, current_counter

In [25]:
sleep_time = 0.5
go_time = 0.15

time.sleep(sleep_time)
x=0.5

target_msg, current_counter= create_message()

b.send_msg('franka_target_pos', target_msg)
time.sleep(sleep_time)


msg = b.recv_msg("franka_state", 0)
print(msg.get_fnumber())

A=np.array([ 0,0,0,0.0825,-0.0825,0,0.088,0])
D=np.array([ 0.333,0,0.316,0,0.384,0,0,0.107])
Alph=(np.pi/2)*(np.array([0,-1,1,1,-1,1,1,0]))
Thet=np.zeros(8)
#Thet[:7] = np.nan
d = {'a':A, 'd':D,'alpha':Alph, 'theta': Thet}
df = pd.DataFrame(d)

inside_range = True
i = 10
polarity = 1
scaling=1.2

position_file = 'toolbox/joint_positions.txt'
lidar_file = 'toolbox/lidar_positions.txt'
lidar_df = pd.read_csv('lidar_mapping.txt', sep = ',')

while inside_range : 
    timeth=i/scaling


    y, z = create_spiral(timeth, 0.005)
    pos = np.array([x, y, z+0.7])
    frame=current_counter+1
    target_msg, current_counter = create_message(counter=frame, pos = pos, go_time=go_time*0.3*i)

    
    msg = b.recv_msg("franka_state", 0)
    Thet[0:7] = msg.get_j_pos()
    df['theta'] = Thet
    end_effect_pos = msg.get_c_pos()
    
    trans_mats = km.Transform2Base(df)
    with open(position_file, 'w') as pf:
        for ourmatrix in trans_mats:
            end_pos = np.zeros(4)
            end_pos[3] = 1
            ep = np.round(np.dot(ourmatrix , end_pos), 3)
            for k in ep: 
                pf.write(str(k) + ',')
        pf.write('\n')
        
    with open(lidar_file, 'w') as lf: 
        l_pos = np.ones(4)
        
        for j in range(len(lidar_df)): 
            l_pos[0] = lidar_df.loc[j].values[2]*0.01
            l_pos[1] = lidar_df.loc[j].values[3]*0.01
            l_pos[2] = lidar_df.loc[j].values[4]*0.01
            #l_pos[:3] = df.loc[i].values[2:]
            lider_joint = lidar_df.loc[j][1]
            lp = np.round(np.dot(trans_mats[int(lider_joint)] , l_pos), 3)
            if j == 0: 
                first_lidar_pos = lp
            for k in lp: 
                lf.write(str(k) + ',')
        lf.write('\n')
        
        
        
        
    lidar_msg = b.recv_msg("franka_lidar", 0)
    lid = lidar_msg.get_data()
    

    
    clear_output(wait = True)
    print(pos)
    print(polarity)
    b.send_msg('franka_target_pos', target_msg)
    time.sleep(sleep_time)
    i += polarity
    
    if z > 0.2: 
        polarity = -1
    elif i == 10: 
        polarity = 1

        
    if lid[0] < 200: 
        print('something approaching. I will go home!')
        escape_pos = first_lidar_pos[:3]
        #escape_pos[2] -= 0.2
        #escape_pos[0] += 0.3
        escape_pos[1] -= lid[0]*0.001
        target_msg, current_counter = create_message(counter=frame +1, pos = escape_pos, go_time=go_time*0.3*i)
        b.send_msg('franka_target_pos', target_msg)
        time.sleep(sleep_time)
        inside_range = False


63813


Exception: Signal franka_lidar is unknown. Is it registered as input?

In [ ]:
timestamp       : Timestamp
fnumber         : uint
c_acc_mean      : [double]  // (3x1)    Mean values of desired end effector
                                        acceleration in cartesion space
c_acc_var       : [double]  // (3x1)    Variance values of desired end 
                                        effector acceleration in cartesian 
                                        space
c_ori_mean      : [double]  // (3x1)    Mean values of desired end effector
                                        orientation in cartesion space
c_ori_var       : [double]  // (3x1)    Variance values of desired end 
                                        effector orientation in cartesian 
                                        space


In [ ]:
gripper_cmd

timestamp       : Timestamp
fnumber         : uint
cmd_t           : enum      //          Encoding for the command type
                                        [UNK=-1, Homing, Grasp, Move, Stop]
width           : double    //          Target Width of the gripper [m]  
speed           : double    //          Target closing speed [m/s]
force           : double    //          Target closing force [N]
epsilon_in      : double    //          Inner tolerance for final width when 
                                        grasped [m]
epsilon_out     : double    //          Inner tolerance for final width when 
                                        grasped [m]

In [ ]:
b.register_signal('task_policy', pab.MsgType.task_policy)

In [3]:
b.register_signal('franka_gripper', pab.MsgType.gripper_cmd)

True

In [ ]:
i = 0
sleep_time = 1
while True: 
    counter = 2000 + 5*i
    if i%2 == 0: 
        ori = np.ones(3)*0.5
    else: 
        ori = np.array([0.5, 0.5, 0])
    
    var = np.ones(3)*0.2
    gripper_msg = pab.task_policy_msg()
    gripper_msg.set_timestamp(time.CLOCK_MONOTONIC)
    gripper_msg.set_fnumber(counter)
    gripper_msg.set_c_ori_mean(ori)
    gripper_msg.set_c_ori_var(var)
    
    gripper_msg.set_c_acc_mean(np.zeros(3))
    gripper_msg.set_c_acc_var(var)
    b.send_msg('task_policy', gripper_msg)
    time.sleep(sleep_time)
    
    print(ori)
    i += 1 

In [2]:
i = 0
sleep_time = 1
while True: 
    counter = 2000 + 5*i
    if i%2 == 0: 
        ori = np.ones(3)*0.5
    else: 
        ori = np.array([0.5, 0.5, 0])
    
    var = np.ones(3)*0.2
    gripper_msg = pab.gripper_cmd_msg()
    gripper_msg.set_timestamp(time.CLOCK_MONOTONIC)
    gripper_msg.set_fnumber(counter)
    gripper_msg.set_cmd_t(2)
    gripper_msg.set_width(0.1)
    gripper_msg.set_speed(0.1)
    #gripper_msg.set_force(0.1)
    gripper_msg.set_epsilon_in(0.005)
    gripper_msg.set_epsilon_out(0.005)
    
    b.send_msg('franka_gripper', gripper_msg)
    time.sleep(sleep_time)
    
    print(ori)
    i += 1 

AssertionError: 

In [ ]:
sleep_time = 0.5
go_time = 0.15

time.sleep(sleep_time)
x=0.5

target_msg, current_counter= create_message()

b.send_msg('franka_target_pos', target_msg)
time.sleep(sleep_time)


msg = b.recv_msg("franka_state", 0)
print(msg.get_fnumber())

A=np.array([ 0,0,0,0.0825,-0.0825,0,0.088,0])
D=np.array([ 0.333,0,0.316,0,0.384,0,0,0.107])
Alph=(np.pi/2)*(np.array([0,-1,1,1,-1,1,1,0]))
Thet=np.zeros(8)
#Thet[:7] = np.nan
d = {'a':A, 'd':D,'alpha':Alph, 'theta': Thet}
df = pd.DataFrame(d)

inside_range = True
i = 10
polarity = 1
scaling=1.2

position_file = 'toolbox/joint_positions.txt'
lidar_file = 'toolbox/lidar_positions.txt'
lidar_df = pd.read_csv('lidar_mapping.txt', sep = ',')

while inside_range : 
    timeth=i/scaling